In [1]:
#%tensorflow_version 2.x
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style='whitegrid')

from IPython.display import clear_output
from six.moves import urllib

import tensorflow.compat.v2.feature_column as fc

import tensorflow as tf

In [2]:
import os
for dirname, _, filenames in os.walk("input"):
    for filename in filenames:
        print(os.path.join(dirname, filename))

input\ks-projects-201612.csv
input\ks-projects-201801.csv


In [3]:
prj2018 = pd.read_csv("input\ks-projects-201801.csv")

In [4]:
prj2018.pop('name')
prj2018.pop('ID')
prj2018.pop('usd pledged')

0            0.0
1          100.0
2          220.0
3            1.0
4         1283.0
           ...  
378656      25.0
378657     155.0
378658      20.0
378659     200.0
378660     524.0
Name: usd pledged, Length: 378661, dtype: float64

In [5]:
deadlines = prj2018.pop('deadline')
launches = prj2018.pop('launched')
deadlines.head()

0    2015-10-09
1    2017-11-01
2    2013-02-26
3    2012-04-16
4    2015-08-29
Name: deadline, dtype: object

In [6]:
count = 0
new_data = pd.DataFrame({'time':int()},index = deadlines.index, columns = ['time'],dtype=int) 
for x in deadlines:
    temp_str_1 = str(deadlines.iloc[count])
    temp_str_2 = str(launches.iloc[count])
    
    temp_int_1_y = int(temp_str_1[0:4])
    temp_int_2_y = int(temp_str_2[0:4])
    temp_int_1_m = int(temp_str_1[5:7])
    temp_int_2_m = int(temp_str_2[5:7])
    temp_int_1_d = int(temp_str_1[8:])
    temp_int_2_d = int(temp_str_2[8:10])
    
    temp_int = (temp_int_1_y - temp_int_2_y)*365 + (temp_int_1_m - temp_int_2_m)*30 + (temp_int_1_d - temp_int_2_d) + 1.0 - 1.0
    #new_data.loc[count] = np.float64(temp_int)
    new_data.loc[count] = temp_int
    count += 1
new_data.head()

,time
0,58.0
1,59.0
2,44.0
3,29.0
4,55.0


In [7]:
prj2018 = prj2018.join(new_data)

In [8]:
prj2018 = prj2018.query('state != "live"')

In [9]:
states = pd.DataFrame({'state':int()},index = prj2018.index, columns = ['state'],dtype=int)
states.loc[prj2018['state'] == "successful", 'state'] = 1
states.loc[prj2018['state'] != "successful", 'state'] = 0

In [10]:
prj2018.pop('state')
prj2018 = prj2018.join(states)

In [11]:
x_prj2018_trn = prj2018.iloc[:340795,:]
x_prj2018_eval = prj2018.iloc[340795:,:]

In [12]:
y_prj2018_trn = x_prj2018_trn.pop('state')
y_prj2018_eval = x_prj2018_eval.pop('state')

In [14]:
CATEGORICAL_COLUMNS = ['category', 'main_category', 'currency', 'country','backers']
NUMERIC_COLUMNS = ['goal', 'pledged','usd_pledged_real','usd_goal_real','time']

feature_columns_1 = []
for feature_name in CATEGORICAL_COLUMNS:
  vocabulary = x_prj2018_trn[feature_name].unique()  # gets a list of all unique values from given feature column
  categorical_column = tf.feature_column.categorical_column_with_vocabulary_list(feature_name, vocabulary,default_value=0)
  feature_columns_1.append(tf.feature_column.indicator_column(categorical_column))
  print(categorical_column)
for feature_name in NUMERIC_COLUMNS:
  feature_columns_1.append(tf.feature_column.numeric_column(feature_name, dtype=tf.float64))

#print(feature_columns_1)

VocabularyListCategoricalColumn(key='category', vocabulary_list=('Poetry', 'Narrative Film', 'Music', 'Film & Video', 'Restaurants', 'Food', 'Drinks', 'Product Design', 'Documentary', 'Nonfiction', 'Indie Rock', 'Crafts', 'Games', 'Tabletop Games', 'Design', 'Comic Books', 'Art Books', 'Fashion', 'Childrenswear', 'Theater', 'Comics', 'DIY', 'Webseries', 'Animation', 'Food Trucks', 'Public Art', 'Illustration', 'Photography', 'Pop', 'People', 'Art', 'Family', 'Fiction', 'Accessories', 'Rock', 'Hardware', 'Software', 'Weaving', 'Gadgets', 'Web', 'Jazz', 'Ready-to-wear', 'Festivals', 'Video Games', 'Anthologies', 'Publishing', 'Shorts', 'Electronic Music', 'Radio & Podcasts', 'Apps', 'Cookbooks', 'Apparel', 'Metal', 'Comedy', 'Hip-Hop', 'Periodicals', 'Dance', 'Technology', 'Painting', 'World Music', 'Photobooks', 'Drama', 'Architecture', 'Young Adult', 'Latin', 'Mobile Games', 'Flight', 'Fine Art', 'Action', 'Playing Cards', 'Makerspaces', 'Punk', "Children's Books", 'Audio', 'Performanc

In [29]:
def input_fn(features, labels, training=True, batch_size=256):
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    # Shuffle and repeat if you are in training mode.
    if training:
        dataset = dataset.shuffle(1000).repeat()
    
    return dataset.batch(batch_size)

In [30]:
classifier = tf.estimator.DNNClassifier(
    feature_columns=feature_columns_1,
    # Two hidden layers of 30 and 10 nodes respectively.
    hidden_units=[30, 10],
    # The model must choose between 3 classes.
    n_classes=2)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\KAAN_A~1\\AppData\\Local\\Temp\\tmpahc0buef', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [32]:
classifier.train(input_fn=lambda: input_fn(x_prj2018_trn, y_prj2018_trn, training=True),steps=50000)

INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\KAAN_A~1\AppData\Local\Temp\tmpahc0buef\model.ckpt-5000
Instructions for updating:
Use standard file utilities to get mtimes.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 5000...
INFO:tensorflow:

In [33]:
eval_result = classifier.evaluate(
    input_fn=lambda: input_fn(x_prj2018_eval, y_prj2018_eval, training=False))

print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-09-24T21:23:25Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\KAAN_A~1\AppData\Local\Temp\tmpahc0buef\model.ckpt-5050
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 2.07780s
INFO:tensorflow:Finished evaluation at 2020-09-24-21:23:27
INFO:tensorflow:Saving dict for global step 505